<a href="https://colab.research.google.com/github/cars1015/recsys_MyReserch/blob/main/EASE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# Googleドライブをマウント
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from scipy.sparse import csr_matrix
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm


class EASE:
    def __init__(self):
        self.user_enc = LabelEncoder()
        self.item_enc = LabelEncoder()

    def _get_users_and_items(self, df):
        users = self.user_enc.fit_transform(df.loc[:, 'uid'])
        items = self.item_enc.fit_transform(df.loc[:, 'sid'])
        return users, items

    def fit(self, df, lambda_: float = 0.5, implicit=True):
        """
        df: pandas.DataFrame with columns user_id, item_id and (rating)
        lambda_: l2-regularization term
        implicit: if True, ratings are ignored and taken as 1, else normalized ratings are used
        """
        users, items = self._get_users_and_items(df)
        values = np.ones(df.shape[0]) if implicit else df['rating'].to_numpy() / df['rating'].max()

        X = csr_matrix((values, (users, items)))
        self.X = X

        G = X.T.dot(X).toarray()
        diagIndices = np.diag_indices(G.shape[0])
        G[diagIndices] += lambda_
        P = np.linalg.inv(G)
        B = P / (-np.diag(P))
        B[diagIndices] = 0

        self.B = B
        # self.pred = X.dot(B)


In [ ]:
data = "ml-20m"
dir = "/content/drive/MyDrive/recommend/" + data + "/pro_sg/"

import pandas as pd

df = pd.read_csv(dir + "train.csv")
# df = pd.read_csv(dir + "train-editor-ml-20m-movie.csv")

model = EASE()
model.fit(df, lambda_ = 500.0)


In [ ]:
df_test = pd.read_csv(dir + "test_tr.csv")

#
users = df_test.loc[:, 'uid']
items = df_test.loc[:, 'sid']

u_enc = LabelEncoder()
users_id = u_enc.fit_transform(users)
items_id = model.item_enc.transform(items)

# 
values = np.ones(df_test.shape[0])
shape = (u_enc.classes_.size, model.item_enc.classes_.size)
X = csr_matrix((values, (users_id, items_id)), shape=shape)

#
pred = X.dot(model.B)

In [ ]:
df_results = pd.DataFrame()

dd_test = df_test.groupby('uid')

k=100

item_set = model.item_enc.classes_

for user, group in tqdm(dd_test):
  watched = set(group['sid'])
  candidates = [item for item in item_set if item not in watched]
  uid = u_enc.transform([user])
  p = np.take(pred[uid[0], :], candidates)
  res = np.argpartition(p, -k)[-k:]
  r = pd.DataFrame({
                "uid": [user] * len(res),
                "sid": np.take(candidates, res),
                "score": np.take(p, res)
      }).sort_values('score', ascending=False)
  #df_results = df_results.append(r, ignore_index=True)
  pd.concat([r, df_results])

df_results.to_csv(dir + "ease_results_500.csv", index=False, header=False)

100%|██████████| 10000/10000 [00:57<00:00, 175.22it/s]
